In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url_list = [f'https://www.khug.or.kr/jeonse/web/s07/s070102.jsp?cur_page={i}' for i in range(1, 61)]

# url에서 가져온 html에서 class_name이 'mb d_list'인 table을 찾아서 df로 만들고 리스트를 concat\
df_list = []
for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    for i, tr in enumerate(table.find_all('tr')):
        if i == 0:
            columns = [th.text for th in tr.find_all('th')]+['안심전세포털']
            df_tmp = pd.DataFrame(columns=columns)
        else:
            values = [td.text for td in tr.find_all('td')]
            link = 'https://www.khug.or.kr/jeonse/web/s07/'+tr.find('a')['href']
            values.append(link)
            df_tmp = pd.concat([df_tmp, pd.DataFrame([values], columns=columns)], ignore_index=True)
    df_list.append(df_tmp)

df = pd.concat(df_list, ignore_index=True)
df.drop(columns=['청약 접수기간', '번호', '공고일자','신청자수'], inplace=True)
df = df.applymap(lambda x: x.replace('  ', '').strip())
df['네이버지도'] = df['주소'].apply(lambda x: f'https://map.naver.com/p/search/{x}')

In [ ]:
df.to_csv('rent_house_list.csv', index=False, encoding='utf-8-sig')